# Customize vanilla MONAILabel `radiology` app
In this document, we will review how to install MONAILabel, install the default (vanilla) `radiology` app, and customize the segmentation app for our needs.

## Structure of vanilla MONAILabel `radiology` app
**Please refer to the included `dicom2deployment-initial-setup.ipynb` notebook for instructions on installing the MONAILabel radiology app.**

The folder structure of the vanilla MONAILabel app will have the following directory tree. We start by editing the default segmentation.py config file shown below.

```
apps
└── radiology
    └── bin
    └── lib
        └── configs
            ├── segmentation.py
            ...
        └── infers
            ...
        └── trainers
            ...
    └── model
        ...
    ...
```

## Customization
For this project, we want to customize the segmentation app to do a multi-label segmentation of the eyeballs (globes) from head CT images. The following subsections briefly introduce the customization of the supporting scripts for the radiology segmentation app. 

### Copying the segmentation config
First we will copy `./apps/radiology/lib/configs/segmentation.py` to `./apps/radiology/lib/configs/segmentation_globes.py` so that we don't have to change the original file.

In [1]:
import shutil

_ = shutil.copyfile('./apps/radiology/lib/configs/segmentation.py', './apps/radiology/lib/configs/segmentation_globes.py')

### Customize labels

Now that we have a new config file to edit, we will change the expected labels. Open the newly created `./apps/radiology/lib/configs/segmentation.py` file in your favorite python editor. The vanilla segmentation config includes the following labels:

```python
# Labels
self.labels = {
    "spleen": 1,
    "kidney_right": 2,
    "kidney_left": 3,
    "gallbladder": 4,
    "liver": 5,
    "stomach": 6,
    "aorta": 7,
    "inferior_vena_cava": 8,
    "portal_vein_and_splenic_vein": 9,
    "pancreas": 10,
    "adrenal_gland_right": 11,
    "adrenal_gland_left": 12,
    "lung_upper_lobe_left": 13,
    "lung_lower_lobe_left": 14,
    "lung_upper_lobe_right": 15,
    "lung_middle_lobe_right": 16,
    "lung_lower_lobe_right": 17,
    "esophagus": 42,
    "trachea": 43,
    "heart_myocardium": 44,
    "heart_atrium_left": 45,
    "heart_ventricle_left": 46,
    "heart_atrium_right": 47,
    "heart_ventricle_right": 48,
    "pulmonary_artery": 49,
}
```

We will change this to right and left globes for our segmentation config:
```python
# Labels
self.labels = {
            "right_globe": 1,
            "left_globe": 2,
}
```

### Disable pretrained models
MONAILabel has several pretrained models that may be useful for your projects. However, there is currently no pretrained CT model that has exactly 2 labels. For this reason, we will disable the pretrained model download and start from scratch. To accomplish this all we need to do is modify the following line of code:
```python
# Download PreTrained Model
if strtobool(self.conf.get("use_pretrained_model", "true")):
            url = f"{self.conf.get('pretrained_path', self.PRE_TRAINED_PATH)}"
            url = f"{url}/radiology_segmentation_segresnet_multilabel.pt"
            download_file(url, self.path[0])
```
and change it to (only change is `true` to `false` in first line:
```python
# Download PreTrained Model
if strtobool(self.conf.get("use_pretrained_model", "false")):
            url = f"{self.conf.get('pretrained_path', self.PRE_TRAINED_PATH)}"
            url = f"{url}/radiology_segmentation_segresnet_multilabel.pt"
            download_file(url, self.path[0])
```

### Add active learning modules
For the last step of our customization, we will add a few codeblocks that enable calculation of epistemic loss. This will allow us to use active learning to accelerate data labeling in MONAILabel. First we need to add some additional imports at the top of the script:

```python
from monailabel.interfaces.tasks.scoring import ScoringMethod
from monailabel.interfaces.tasks.strategy import Strategy
from monailabel.tasks.activelearning.epistemic import Epistemic
from monailabel.tasks.scoring.dice import Dice
from monailabel.tasks.scoring.epistemic import EpistemicScoring
from monailabel.tasks.scoring.sum import Sum
```

Next, we will add the following lines to the end of the `init` method of the `Segmentation` class.
```python
# for epistemic uncertainty
self.epistemic_enabled = strtobool(conf.get("epistemic_enabled", "false"))
self.epistemic_samples = int(conf.get("epistemic_samples", "5"))
logger.info(f"EPISTEMIC Enabled: {self.epistemic_enabled}; Samples: {self.epistemic_samples}")
```
Next we will add the following two additional methods at the end of the `Segmentation` class:

```python
def strategy(self) -> Union[None, Strategy, Dict[str, Strategy]]:
    strategies: Dict[str, Strategy] = {}
    if self.epistemic_enabled:
        strategies[f"{self.name}_epistemic"] = Epistemic()
    return strategies

def scoring_method(self) -> Union[None, ScoringMethod, Dict[str, ScoringMethod]]:
    methods: Dict[str, ScoringMethod] = {
        "dice": Dice(),
        "sum": Sum(),
    }

    if self.epistemic_enabled:
        methods[f"{self.name}_epistemic"] = EpistemicScoring(
            model=self.path,
            network=self.network,
            transforms=lib.infers.Segmentation(None).pre_transforms(),
            num_samples=self.epistemic_samples,
        )
    return methods
```

# Start the MONAILabel server
Now that we have customized the MONAILabel segmentation config, we can start the MONAILabel server and try it out. In a terminal window, navigate to the directory containing this notebook and run:

```bash
monailabel start_server --app apps/radiology --studies nifti_resampled --conf models segmentation_globes
```
### Explanation of options
* `--app apps/radiology` - this is the location of your radiology app (downloaded earlier in this document). This can be moved to any location you desire.
* `--studies nifti_resampled` - this is the location of your data. In this case, we are using the `nifti_resampled` directory that we created in the `dicom2deployment-data-prep.ipynb` document.
* `--conf models segmentation_globes` - this is the location of our custom segmentation config (created earlier in this notebook).

If we want to enable epistemic loss calculation (for active learning) then we need to add a few more options to the start_server call. This is only relevant once you have already trained at least one initial model:

```bash
monailabel start_server --app apps/radiology --studies nifti_resampled --conf models segmentation_globes --conf skip_scoring false --conf skip_strategies false --conf epistemic_enabled true
```
### Explanation of additional config options for epistemic uncertainty
* `--conf skip_scoring false` - this tells MONAILabel to score each of the unlabeled studies using the `scoring_method` we defined previously.
* `--conf skip_strategies false` - this enables the active learning `strategy` method that we defined previously.
* `--conf epistemic_enabled true` - this enables epistemic scoring.

# Using 3D Slicer to access the MONAILabel server
Now that we are running the MONAILabel server on our local machine, we can access it locally using 3D Slicer. Detailed instructions for installing 3D Slicer and accessing the local server can be found here:

https://docs.monai.io/projects/label/en/latest/quickstart.html#install-monai-label-plugin-in-3d-slicer
# Enabling MONAILabel server on your local area network
The following (optional) steps will allow other people on your local area network to access the MONAILabel server and collaborate on the active learning segmentation task.

## <span style="color:red">CAUTION: SECURITY RISK</span>
<span style="color:red">Please do not attempt the follow steps without consulting an IT/networking professional. The following steps, particularly opening a port, can expose your computer to unauthorized access.</span>

## Opening a port
By default, MONAILabel runs on port 8000. This can be changed using the `--port` flag to the `start_server` command above. To open this port, you can use one of the following methods:
* Ubuntu-based Linux: `sudo ufw allow 4000`
* Cent-OS-based Linux: `firewall-cmd --add-port=4000/tcp`
* Windows: https://learn.microsoft.com/en-us/sql/reporting-services/report-server/configure-a-firewall-for-report-server-access?view=sql-server-ver16 (use port 8000 instead of port 80 as written)
* MacOS: https://www.macworld.com/article/671729/mac-firewall-how-to-open-specific-ports-in-os-x-10-10-firewall.html (use port 8000)

## Determine your private IP
In order for others in your local area network to access your MONAILabel server, you will need to provide your private IP address. This can be found on most systems using the `ipconfig` or `ifconfig` commands. You are looking for something like this: `inet 192.168.86.25`. 

## Connecting to the MONAILabel server over the local network
Using the private IP address from above example, users would be able to connect to your MONAILabel server using the following address:

`http://192.168.86.25:8000`

![Server IP Entry](img/slicer.jpeg)

# Deploying a MONAILabel server on AWS
Another easier and more secure method for deploying a MONAILabel server is the AWS cloud. AWS has pre-built solutions for scalable MONAILabel deployment. For more information please see:

https://aws.amazon.com/blogs/industries/ai-assisted-annotation-of-medical-images-using-monai-label-on-aws/

If you are interested, you can get in touch with an AWS sales representative here:

https://pages.awscloud.com/HealthContactSales.html?languages=english

## Finished!
Congratulations! You have completed the MONAILabel section.